In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import statistics
from collections import Counter
import pyarrow as pa
import os
from collections import OrderedDict
import scipy.stats
sns.set_style("whitegrid")

In [2]:
tempTopDir = '/Volumes/Britt_SSD/ReDoEvals3/UserStudy/'
filePath = os.path.join(tempTopDir, 'GP_Main.parquet')

In [3]:
taskDF = pd.read_parquet(filePath)
taskDF['ParticipantNum'] = taskDF["ParticipantPrivateID"]
taskDF = taskDF.astype({"ParticipantPrivateID": str})
filePath = os.path.join(tempTopDir, 'GP_Main_Short.parquet')
slimDF = pd.read_parquet(filePath)
print(len(slimDF))

249


In [4]:
# randomiser_k74v = LVL
# randomiser_8rx4 = VisCond_DIFF
# randomiser_zcqq = VisCond_EASY


# randomiser_ndwh = AgentCond_BlurVis_DIFF
# randomiser_7vb5 = AgentCond_Blur_DIFF
# randomiser_fy7r = AgentCond_Vis_DIFF
# randomiser_vfim = AgentCond_Norm_DIFF

# randomiser_l6v8 = AgentCond_BlurVis_EASY
# randomiser_dwc2 = AgentCond_Blur_EASY
# randomiser_xnr7 = AgentCond_Vis_EASY
# randomiser_9z8b = AgentCond_Norm_EASY

In [5]:
print(taskDF.ExperimentVersion.unique())
print(slimDF.ExperimentVersion.unique())

[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]
[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]


In [6]:
lookup = taskDF[["VisCond_DIFF","VisCond_EASY"]].notnull().idxmax(1)
taskDF = taskDF.assign(VisCond=taskDF.lookup(lookup.index, lookup.values))
taskDF['VisCond']

/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  


index
0        Blur_Plus_Vis
1        Blur_Plus_Vis
2        Blur_Plus_Vis
3        Blur_Plus_Vis
4        Blur_Plus_Vis
             ...      
75810          Blurred
75811          Blurred
75812          Blurred
75813          Blurred
75814          Blurred
Name: VisCond, Length: 75676, dtype: object

In [7]:
lookup = taskDF[["AgentCond_BlurVis_DIFF","AgentCond_Blur_DIFF","AgentCond_Vis_DIFF","AgentCond_Norm_DIFF",
                 "AgentCond_BlurVis_EASY","AgentCond_Blur_EASY","AgentCond_Vis_EASY","AgentCond_Norm_EASY"]].notnull().idxmax(1)
taskDF = taskDF.assign(AgentCond=taskDF.lookup(lookup.index, lookup.values))
taskDF['AgentCond']

/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  This is separate from the ipykernel package so we can avoid doing imports until


index
0        300_700
1        300_700
2        300_700
3        300_700
4        300_700
          ...   
75810    300_700
75811    300_700
75812    300_700
75813    300_700
75814    300_700
Name: AgentCond, Length: 75676, dtype: object

In [8]:
filePath = os.path.join(tempTopDir, 'GP_Consent_Long.parquet')
consentDF_L = pd.read_parquet(filePath).infer_objects()
consentDF_L = consentDF_L.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
consentDF_L = consentDF_L[consentDF_L['ParticipantPrivateID'].notna()]

filePath = os.path.join(tempTopDir, 'GP_Consent_Short.parquet')
consentDF_S = pd.read_parquet(filePath).infer_objects()
consentDF_S = consentDF_S.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
consentDF_S = consentDF_S[consentDF_S['ParticipantPrivateID'].notna()]

filePath = os.path.join(tempTopDir, 'GP_Demographics_Long.parquet')
demoDF_L = pd.read_parquet(filePath).infer_objects()
demoDF_L = demoDF_L.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
demoDF_L = demoDF_L[demoDF_L['ParticipantPrivateID'].notna()]

filePath = os.path.join(tempTopDir, 'GP_Demographics_Short.parquet')
demoDF_S = pd.read_parquet(filePath).infer_objects()
demoDF_S = demoDF_S.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
demoDF_S = demoDF_S[demoDF_S['ParticipantPrivateID'].notna()]

filePath = os.path.join(tempTopDir, 'GP_AI_Long.parquet')
aiDF_L = pd.read_parquet(filePath).infer_objects()
aiDF_L = aiDF_L.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
aiDF_L = aiDF_L[aiDF_L['ParticipantPrivateID'].notna()]

filePath = os.path.join(tempTopDir, 'GP_AI_Short.parquet')
aiDF_S = pd.read_parquet(filePath).infer_objects()
aiDF_S = aiDF_S.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
aiDF_S = aiDF_S[aiDF_S['ParticipantPrivateID'].notna()]

In [9]:
demoDF_L.columns

Index(['EventIndex', 'UTCTimestamp', 'UTCDate', 'LocalTimestamp',
       'LocalTimezone', 'LocalDate', 'ExperimentID', 'ExperimentVersion',
       'TreeNodeKey', 'RepeatKey', 'ScheduleID', 'ParticipantPublicID',
       'ParticipantPrivateID', 'ParticipantStartingGroup', 'ParticipantStatus',
       'ParticipantCompletionCode', 'ParticipantExternalSessionID',
       'ParticipantDeviceType', 'ParticipantDevice', 'ParticipantOS',
       'ParticipantBrowser', 'ParticipantMonitorSize',
       'ParticipantViewportSize', 'Checkpoint', 'TaskName', 'TaskVersion',
       'randomiser_8rx4', 'randomiser_ndwh', 'randomiser_fy7r',
       'randomiser_7vb5', 'checkpoint_ph4w', 'checkpoint_drrq',
       'randomiser_vfim', 'checkpoint_6gsl', 'checkpoint_apgi',
       'randomiser_k74v', 'randomiser_zcqq', 'randomiser_9z8b',
       'randomiser_xnr7', 'randomiser_l6v8', 'randomiser_dwc2',
       'Randomisequestionnaireelements?', 'QuestionKey', 'Response',
       'branch_4zei', 'age_info', 'age_info_text', 

In [10]:
aiDF_S.columns

Index(['EventIndex', 'UTCTimestamp', 'UTCDate', 'LocalTimestamp',
       'LocalTimezone', 'LocalDate', 'ExperimentID', 'ExperimentVersion',
       'TreeNodeKey', 'RepeatKey', 'ScheduleID', 'ParticipantPublicID',
       'ParticipantPrivateID', 'ParticipantStartingGroup', 'ParticipantStatus',
       'ParticipantCompletionCode', 'ParticipantExternalSessionID',
       'ParticipantDeviceType', 'ParticipantDevice', 'ParticipantOS',
       'ParticipantBrowser', 'ParticipantMonitorSize',
       'ParticipantViewportSize', 'Checkpoint', 'TaskName', 'TaskVersion',
       'randomiser_8rx4', 'randomiser_ndwh', 'randomiser_fy7r',
       'randomiser_7vb5', 'checkpoint_ph4w', 'checkpoint_drrq',
       'randomiser_vfim', 'checkpoint_6gsl', 'checkpoint_apgi',
       'randomiser_k74v', 'randomiser_zcqq', 'randomiser_9z8b',
       'randomiser_xnr7', 'randomiser_l6v8', 'randomiser_dwc2',
       'Randomisequestionnaireelements?', 'ai_involvement_1',
       'ai_involvement_2', 'ai_involvement_3', 'ai_involve

In [11]:
len(demoDF_L.ParticipantPrivateID.unique())

246

In [12]:
len(slimDF.ParticipantPrivateID.unique())

245

In [13]:
taskDF.columns

Index(['EventIndex', 'UTCTimestamp', 'UTCDate', 'LocalTimestamp',
       'LocalTimezone', 'LocalDate', 'ExperimentID', 'ExperimentVersion',
       'TreeNodeKey', 'RepeatKey', 'ScheduleID', 'ParticipantPublicID',
       'ParticipantPrivateID', 'ParticipantStartingGroup', 'ParticipantStatus',
       'ParticipantCompletionCode', 'ParticipantExternalSessionID',
       'ParticipantDeviceType', 'ParticipantDevice', 'ParticipantOS',
       'ParticipantBrowser', 'ParticipantMonitorSize',
       'ParticipantViewportSize', 'Checkpoint', 'TaskName', 'TaskVersion',
       'VisCond_DIFF', 'AgentCond_BlurVis_DIFF', 'AgentCond_Vis_DIFF',
       'AgentCond_Blur_DIFF', 'checkpoint_ph4w', 'checkpoint_drrq',
       'AgentCond_Norm_DIFF', 'checkpoint_6gsl', 'checkpoint_apgi', 'LVL',
       'VisCond_EASY', 'AgentCond_Norm_EASY', 'AgentCond_Vis_EASY',
       'AgentCond_BlurVis_EASY', 'AgentCond_Blur_EASY', 'Spreadsheet',
       'SpreadsheetName', 'SpreadsheetRow', 'TrialNumber', 'ScreenNumber',
       'Scre

In [14]:
datamap = {
    1: "Extremely negative", 
    2: "Negative", 
    3: "Neutral", 
    4: "Positive", 
    5: "Extremely Positive"
}


aiDF_S['AIrating'] = aiDF_S.ai_opinion_quantised.map(datamap.get)
slimDF['AIrating'] = slimDF.ai_opinion_quantised.map(datamap.get)



In [15]:
datamap = {
    1: "Less than 1 year ago",  
    2: "1 to 5 years ago",  
    3: "More than 5 years ago",  
    4: "Never"
}


aiDF_S['PMrating'] = aiDF_S.pacman_experience_quantised.map(datamap.get)



In [16]:
datamap = {
    1: "M",  
    2: "F",  
    3: "No Answer",  
    4: "Other"
}


demoDF_S['gender_rating'] = demoDF_S.gender_id_quantised.map(datamap.get)



In [17]:
datamap = {
    1: "18 – 20",  
    2: '21 – 30',   
    3: '31 – 40',   
    4: '41 – 50',  
    5: '51 – 60', 
    6: '61 – 70',  
    7: '71+'
}


demoDF_S['age_rating'] = demoDF_S.age_info_quantised.map(datamap.get)


In [18]:

colors = ["blue","darkorange", "#74C476"]
df = demoDF_S.groupby(['age_rating','gender_rating']).sum().unstack().add_prefix('param_')
print(df['param_gender_id_quantised'])
print(df['param_age_info_quantised'])
# df['param_gender_id_quantised'].plot(kind="bar",stacked=True, figsize=(10,7), color=colors)
# plt.show()
# df['param_age_info_quantised'].plot(kind="bar",stacked=True, figsize=(10,7), color=colors)

gender_rating  param_F  param_M  param_No Answer
age_rating                                      
18 – 20           24.0     29.0              NaN
21 – 30           84.0     32.0              3.0
31 – 40           64.0     20.0              NaN
41 – 50           44.0     20.0              NaN
51 – 60           20.0     14.0              NaN
61 – 70            6.0      4.0              NaN
71+                NaN      2.0              NaN
gender_rating  param_F  param_M  param_No Answer
age_rating                                      
18 – 20           12.0     29.0              NaN
21 – 30           84.0     64.0              2.0
31 – 40           96.0     60.0              NaN
41 – 50           88.0     80.0              NaN
51 – 60           50.0     70.0              NaN
61 – 70           18.0     24.0              NaN
71+                NaN     14.0              NaN


In [19]:
datamap = {
    1: "HS",  
    2: "College",   
    3: "Grad",   
    4: "NA",  
    5: "Other"
}


demoDF_S['edu_rating'] = demoDF_S.education_level_quantised.map(datamap.get)



In [20]:
# Calculate Participant's Time Spent on the Questionnaires
# importing Statistics module 
import statistics 

time_list = []
sus_list = []

for p_index, pID in enumerate(demoDF_L.ParticipantPrivateID.unique()):
    temp = demoDF_L[demoDF_L['ParticipantPrivateID']==pID].copy()
    
    temp.set_index('EventIndex')
    if (not pd.isnull(temp.iloc[-1]['UTCTimestamp']) and not pd.isnull(temp.iloc[0]['UTCTimestamp'])):
        timeTaken = (temp.iloc[-1]['UTCTimestamp']-temp.iloc[0]['UTCTimestamp'])/1000
        if(timeTaken>0):
            time_list.append(timeTaken)
        if (timeTaken < 1):
            print(temp.ParticipantPublicID.unique())
            sus_list.append(temp.ParticipantPublicID.unique())
            print(timeTaken)
            
print("Mean of time on Demographics is % s Seconds" 
            % (statistics.mean(time_list)))
print("Standard Deviation of time on Demographics is % s Seconds" 
            % (statistics.stdev(time_list))) 


time_list = []

for p_index, pID in enumerate(aiDF_L.ParticipantPrivateID.unique()):
    temp = aiDF_L[aiDF_L['ParticipantPrivateID']==pID].copy()
    
    temp.set_index('EventIndex')
    if (not pd.isnull(temp.iloc[-1]['UTCTimestamp']) and not pd.isnull(temp.iloc[0]['UTCTimestamp'])):
        timeTaken = (temp.iloc[-1]['UTCTimestamp']-temp.iloc[0]['UTCTimestamp'])/1000
        if(timeTaken>0):
            time_list.append(timeTaken)
        if (timeTaken < 1):
            print(temp.ParticipantPublicID.unique())
            sus_list.append(temp.ParticipantPublicID.unique())
            print(timeTaken)
            
print("Mean of time on AI is % s Seconds" 
            % (statistics.mean(time_list)))
print("Standard Deviation of time on AI is % s Seconds" 
            % (statistics.stdev(time_list))) 


# time_list = []

# for p_index, pID in enumerate(consentDF_L.ParticipantPrivateID.unique()):
#     temp = consentDF_L[consentDF_L['ParticipantPrivateID']==pID].copy()
    
#     temp.set_index('EventIndex')
#     if (not pd.isnull(temp.iloc[-1]['UTCTimestamp']) and not pd.isnull(temp.iloc[0]['UTCTimestamp'])):
#         timeTaken = (temp.iloc[-1]['UTCTimestamp']-temp.iloc[0]['UTCTimestamp'])/1000
#         if(timeTaken>0):
#             time_list.append(timeTaken)
#         if (timeTaken < 2):
#             print(temp.ParticipantPublicID.unique())
#             print(timeTaken)
            
# print("Mean of time on Consent is % s Seconds" 
#             % (statistics.mean(time_list)))
# print("Standard Deviation of Consent is % s Seconds" 
#             % (statistics.stdev(time_list))) 

['5f9fc753be94016201d471fc']
0.0
['5ffd82a2cecb152c32eb051a']
0.0
['5fac647436421623d15dfcf5']
0.0
['5f80341cccd27c01be897e4c']
0.0
['5ec75df0d799ae1c11a8115c']
0.0
['5f7715b6b481f41e390d2bf0']
0.0
['5a7905828fe2dc0001058b16']
0.0
['5f782504951dda38db3ef0b6']
0.0
['5f8dcac25e91d312a21a625f']
0.0
['5fc64277766e2802dd28785d']
0.0
['60021a91b11a381fdefe27e1']
0.0
['5f9b28cdd59a8e3148572a38']
0.0
['5dd5819a535a715242f83192']
0.0
['5c35655f87b777000170d85a']
0.0
['60380b33f37c851738ceb6ea']
0.0
['6030050a5bdba4dd09d142f4']
0.0
['5ebe6d9aad61fb02ea09617f']
0.0
['5fe1fa69e708b14ca8cc4e18']
0.0
['5e1cabad3b577b000c55e660']
0.0
['602bfaf031de924145ea935a']
0.0
['5ecf1fb0eed497344398139d']
0.0
['5ea891c923a85e2eeb157452']
0.0
['572f1c1d3c27e7000f0b31aa']
0.0
['603d970e68faadd150aeabf9']
0.0
['5fc49c6f8e28dc8f9e3124a4']
0.0
['5a6fcd526923df0001ef975f']
0.0
['5f90c32f69122f1cebd1cc0b']
0.0
['5e8e5216775eaa111a50f692']
0.0
['5b27c8c9051c6e00019d0c96']
0.0
['5ea59ae1ae3958000acc6400']
0.0
['5eee7768

In [21]:
# tmp = taskDF[taskDF['Response'].dropna()]

print(taskDF[taskDF['Response'].str.contains('END|VIDEO', na=False)]['Response'].head(55))

index
1                     VIDEO STARTED
2         VIDEO PLAYING EVENT FIRED
3      VIDEO TIMEUPDATE EVENT FIRED
5                     VIDEO STARTED
6         VIDEO PLAYING EVENT FIRED
24                    VIDEO STARTED
25     VIDEO TIMEUPDATE EVENT FIRED
27        VIDEO PLAYING EVENT FIRED
28          VIDEO ENDED EVENT FIRED
30                    VIDEO STARTED
31     VIDEO TIMEUPDATE EVENT FIRED
33        VIDEO PLAYING EVENT FIRED
34        VIDEO PLAYING EVENT FIRED
35        VIDEO PLAYING EVENT FIRED
36        VIDEO PLAYING EVENT FIRED
37        VIDEO PLAYING EVENT FIRED
43                    VIDEO STARTED
44     VIDEO TIMEUPDATE EVENT FIRED
46        VIDEO PLAYING EVENT FIRED
49                    VIDEO STARTED
50     VIDEO TIMEUPDATE EVENT FIRED
52        VIDEO PLAYING EVENT FIRED
53        VIDEO PLAYING EVENT FIRED
54          VIDEO ENDED EVENT FIRED
62                    VIDEO STARTED
63     VIDEO TIMEUPDATE EVENT FIRED
65        VIDEO PLAYING EVENT FIRED
66        VIDEO PLAYIN

In [22]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [23]:
# Toy use of above fxn
print("Time")
print(mean_confidence_interval(slimDF['timeTakenMain_All']))
print("1")
print(mean_confidence_interval(slimDF['tot1Perc']))
print("2")
print(mean_confidence_interval(slimDF['tot2Perc']))

Time
(1618.101080321285, 1525.6595152880445, 1710.5426453545256)
1
(27.10843373493976, 24.808488107057418, 29.4083793628221)
2
(27.40963855421687, 25.1186055808148, 29.700671527618937)


In [24]:
print("All main")
print(slimDF['timeTakenMain_All'].mean())
print(slimDF['timeTakenMain_All'].std())

print((slimDF['timeTakenMain_All'].mean() - (2 * slimDF['timeTakenMain_All'].std()))/60)

print("Agents main")
print(slimDF['timeTakenMain_AgentsOnly'].mean())
print(slimDF['timeTakenMain_AgentsOnly'].std())

print((slimDF['timeTakenMain_AgentsOnly'].mean() - (2 * slimDF['timeTakenMain_AgentsOnly'].std()))/60)

print("All")
print(slimDF['time_TOTAL'].mean())
print(slimDF['time_TOTAL'].std())

print((slimDF['time_TOTAL'].mean() - (2 * slimDF['time_TOTAL'].std()))/60)

All main
1618.101080321285
740.6180560331514
2.2810828042497024
Agents main
1205.1329718875502
582.3776055475325
0.6729626798747518
All
1867.7670080321284
858.5552177779726
2.5109428746030518


In [25]:
taskDF.likertCatVar = pd.Categorical(taskDF.Response, categories=['Very False','Somewhat False','Neutral','Somewhat True','Very True'], ordered=True)
taskDF.confCatVar = pd.Categorical(taskDF.Response, categories=['Not at All Confident','Not Confident','Neutral','Somewhat Confident','Completely Confident'], ordered=True)


/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.
/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [26]:
d = {'Very False':1,'Somewhat False':2,'Neutral':3,'Somewhat True':4,'Very True':5,
     'Not at All Confident':1,'Not Confident':2,'Neutral':3,'Somewhat Confident':4,'Completely Confident':5}


taskDF['task_rating'] = taskDF.Response.map(d.get)

In [27]:
# more work on finding sus
sus = ['5d2a1ea859143a0019d3cc60']

for p in taskDF.ParticipantPublicID.unique():
    tempTask = taskDF[taskDF['ParticipantPublicID'] == p]
    tempSlim = slimDF[slimDF['ParticipantPublicID'] == p]

    try:
        if (tempSlim.iloc[0]['timeTakenMain_All']/60) < 7:
            print("Main")
            print(tempSlim.iloc[0]['timeTakenMain_All']/60)
            sus_list.append(p)
    except:
        sus_list.append(p) 

        #     print("Task")
#     print(tempSlim['timeTakenMain_AgentsOnly']/60)
    tempAI = aiDF_L[aiDF_L['ParticipantPublicID']==p]


print(sus_list)

Main
6.80605
Main
6.1482833333333335
[array(['5f9fc753be94016201d471fc'], dtype=object), array(['5ffd82a2cecb152c32eb051a'], dtype=object), array(['5fac647436421623d15dfcf5'], dtype=object), array(['5f80341cccd27c01be897e4c'], dtype=object), array(['5ec75df0d799ae1c11a8115c'], dtype=object), array(['5f7715b6b481f41e390d2bf0'], dtype=object), array(['5a7905828fe2dc0001058b16'], dtype=object), array(['5f782504951dda38db3ef0b6'], dtype=object), array(['5f8dcac25e91d312a21a625f'], dtype=object), array(['5fc64277766e2802dd28785d'], dtype=object), array(['60021a91b11a381fdefe27e1'], dtype=object), array(['5f9b28cdd59a8e3148572a38'], dtype=object), array(['5dd5819a535a715242f83192'], dtype=object), array(['5c35655f87b777000170d85a'], dtype=object), array(['60380b33f37c851738ceb6ea'], dtype=object), array(['6030050a5bdba4dd09d142f4'], dtype=object), array(['5ebe6d9aad61fb02ea09617f'], dtype=object), array(['5fe1fa69e708b14ca8cc4e18'], dtype=object), array(['5e1cabad3b577b000c55e660'], dtype=ob

In [28]:
pd.set_option('display.max_columns', 500)# note, from v68 +, videos auto-play so min is ~53
# get info on a participant:
# w1ue
pID = '5d2a1ea859143a0019d3cc60'
tempT = taskDF[taskDF['ParticipantPublicID']==pID]
tempS = slimDF[slimDF['ParticipantPublicID']==pID]

print(tempT[['ReactionTime', 'Correct','Incorrect', 'Dishonest']].agg(['sum', 'mean','std']))
print("\n")
print(tempT.query('ZoneType=="response_text_area"')[['Response']])
print("\n")

print(tempS[['ParticipantPrivateID', 'difficulty', 'visuals', 'agents', 'QsTotal', 'QsCorrect', 
             'totPerc', 'predPerc', 'regPerc',  'lvlPerc', 'ghostPerc',  'compPerc',
 'timeTakenMain_All', 'timeTakenMain_AgentsOnly', 'timeTaken_Consent',
'timeTaken_AI', 'time_TOTAL', 'vidPlayed', 'avgVidPlays',  'age_info_quantised', 'gender_id_quantised',
 'education_level_quantised', 'ExperimentVersion',  'ClassOnAI', 'DevelopAI', 'AIrating']])

      ReactionTime    Correct   Incorrect  Dishonest
sum   2.717945e+06  12.000000  319.000000        0.0
mean  8.211313e+03   0.036254    0.963746        0.0
std   4.832402e+04   0.187204    0.187204        0.0


                                    Response
index                                       
57826  because I think he's the best player.
57836                                   none
57864                    Because I like him.
57874                                   none
57902                 he is the best player.
57912                                   none
57940                            I like him.
57950                                   none
57958                                   None


      ParticipantPrivateID difficulty visuals   agents  QsTotal  QsCorrect  \
index                                                                        
200              3531762.0  Difficult   Video  s23_p23       20        3.0   

       totPerc   predPerc  regPerc  lvlPerc  ghostPerc

In [29]:
from collections import Counter

for vs_index, version in enumerate(taskDF.ExperimentVersion.unique()):
    print("\nresetting list")
    exp_list = []
    temp1 = taskDF[taskDF['ExperimentVersion']==version]
    for p_index, pID in enumerate(temp1.ParticipantPublicID.unique()):
        temp2 = temp1[temp1['ParticipantPublicID']==pID]
        if len(temp2) > 0:
            temp3 = temp2[temp2['Response']=='VIDEO PLAYING EVENT FIRED']
            exp_list.append(len(temp3))
            if (temp3.ExperimentVersion.unique()[0]<70):
                if (len(temp3)<4):
                    print(pID)
                    sus_list.append(pID)
                    print("Temp3 length")
                    print(len(temp3))
                    print("time:")
                    print(slimDF[slimDF['ParticipantPublicID']==pID]['time_TOTAL'])
            else:
                if (len(temp3)<4):
                    print(pID)
                    sus_list.append(pID)
                    print("Temp3 length")
                    print(len(temp3))
            
    if (len(exp_list) > 0):
        print("for " + str(version) + ": ")
        print(sorted(exp_list))
        print("Avg: " + str(sum(exp_list)/len(exp_list)))
        print("Median: ")
        print(statistics.median(exp_list))
        print("Min: ")
        print(min(exp_list))
        print("Mode:")
        c = Counter(exp_list)
        print(c.most_common(1))


resetting list
for 60.0: 
[31, 61, 62]
Avg: 51.333333333333336
Median: 
61
Min: 
31
Mode:
[(31, 1)]

resetting list
for 62.0: 
[51]
Avg: 51.0
Median: 
51
Min: 
51
Mode:
[(51, 1)]

resetting list
for 64.0: 
[53]
Avg: 53.0
Median: 
53
Min: 
53
Mode:
[(53, 1)]

resetting list
for 65.0: 
[195]
Avg: 195.0
Median: 
195
Min: 
195
Mode:
[(195, 1)]

resetting list
60342dbfcf15a5092cda30dc
Temp3 length
3
time:
index
83    429.475
Name: time_TOTAL, dtype: float64
for 67.0: 
[3, 5, 8, 9, 10, 10, 10, 10, 11, 11, 11, 12, 14, 15, 15, 17, 18, 19, 26, 26, 26, 27, 27, 28, 28, 28, 29, 29, 30, 30, 32, 33, 36, 37, 39, 41, 41, 42, 46, 46, 46, 48, 51, 51, 52, 52, 54, 56, 57, 57, 59, 60, 60, 61, 63, 64, 64, 65, 65, 66, 68, 70, 71, 74, 74, 77, 78, 80, 87, 96, 99, 101, 102, 104, 110, 117, 131, 139, 154, 183, 248, 347, 648]
Avg: 64.3855421686747
Median: 
48
Min: 
3
Mode:
[(10, 4)]

resetting list
for 70.0: 
[60, 68, 88, 92, 104]
Avg: 82.4
Median: 
88
Min: 
60
Mode:
[(60, 1)]

resetting list
for 72.0: 
[57, 58, 

In [30]:
sus = sus_list
print(sus)


[array(['5f9fc753be94016201d471fc'], dtype=object), array(['5ffd82a2cecb152c32eb051a'], dtype=object), array(['5fac647436421623d15dfcf5'], dtype=object), array(['5f80341cccd27c01be897e4c'], dtype=object), array(['5ec75df0d799ae1c11a8115c'], dtype=object), array(['5f7715b6b481f41e390d2bf0'], dtype=object), array(['5a7905828fe2dc0001058b16'], dtype=object), array(['5f782504951dda38db3ef0b6'], dtype=object), array(['5f8dcac25e91d312a21a625f'], dtype=object), array(['5fc64277766e2802dd28785d'], dtype=object), array(['60021a91b11a381fdefe27e1'], dtype=object), array(['5f9b28cdd59a8e3148572a38'], dtype=object), array(['5dd5819a535a715242f83192'], dtype=object), array(['5c35655f87b777000170d85a'], dtype=object), array(['60380b33f37c851738ceb6ea'], dtype=object), array(['6030050a5bdba4dd09d142f4'], dtype=object), array(['5ebe6d9aad61fb02ea09617f'], dtype=object), array(['5fe1fa69e708b14ca8cc4e18'], dtype=object), array(['5e1cabad3b577b000c55e660'], dtype=object), array(['602bfaf031de924145ea93

In [31]:

for vs in taskDF.ExperimentVersion.unique():
    temp = taskDF[taskDF['ExperimentVersion']==vs]
    for tnk in temp.TreeNodeKey.unique():
        temp1 = temp[temp['TreeNodeKey']==tnk]
#         print(temp1['ParticipantPublicID'].tolist())
        for pID in sus:
#             print(pID)
            if pID in temp1['ParticipantPublicID'].tolist():
                print(temp1.ExperimentVersion.unique()[0])
                print(tnk)
                print(pID)

67.0
task-nwyp
5fcb5f768c4bb789f899288f
67.0
task-sh5l
60342dbfcf15a5092cda30dc
67.0
task-sh5l
60342dbfcf15a5092cda30dc
76.0
task-149o
['6030050a5bdba4dd09d142f4']
76.0
task-149o
['5ea59ae1ae3958000acc6400']
76.0
task-17xv
['603d970e68faadd150aeabf9']
76.0
task-17xv
['5b27c8c9051c6e00019d0c96']
76.0
task-1z9u
['5a7905828fe2dc0001058b16']
76.0
task-1z9u
['5f8dcac25e91d312a21a625f']
76.0
task-29ca
['5ecf1fb0eed497344398139d']
76.0
task-29ca
['5f90c32f69122f1cebd1cc0b']
76.0
task-3gz3
['5d2ca342f91bdd001ae993e6']
76.0
task-3r4f
['5f7715b6b481f41e390d2bf0']
76.0
task-4hv7
['5e1cabad3b577b000c55e660']
76.0
task-71l3
['5f9fc753be94016201d471fc']
76.0
task-75su
['572f1c1d3c27e7000f0b31aa']
76.0
task-8t45
['5ea891c923a85e2eeb157452']
76.0
task-8t45
['5c0076b35b01360001fca870']
76.0
task-aijr
['5bfbff5763fc010001526679']
76.0
task-bxhf
['5dd5819a535a715242f83192']
76.0
task-bxhf
['602f83b750c603c79ae6e80d']
76.0
task-dncm
['5fe1fa69e708b14ca8cc4e18']
76.0
task-dncm
['5ea824f580b9ef228f589930']


In [33]:
############################################# STEP ONE #############################################

## Now check participants to see whose data we need to investigate
sus_list = []
# for each participant.... Get number correct, all times spent, and number of times vid was played

for p_index, pID in enumerate(slimDF.ParticipantPublicID.unique()):
    
    temp = pd.DataFrame()
    temp = slimDF[slimDF['ParticipantPublicID']==pID]
#     if (temp.iloc[0]['ExperimentVersion'] == '76.'):
    
#     print("Experiment version: " + str(temp.iloc[0]['ExperimentVersion']))
    v_temp = slimDF[slimDF['visuals']==temp.iloc[0]['visuals']]

#     a = temp.iloc[0]['QsCorrect']
#     b = v_temp.QsCorrect.mean()
#     c = v_temp.QsCorrect.std()
#     if (a < (b-c)):
#         print("Correct:")
#         print(a)
#         print("Avg Correct:")
#         print(b)

#     a = temp.iloc[0]['predictionsCorrect']
#     b = v_temp.predictionsCorrect.mean()
#     c = v_temp.predictionsCorrect.std()
#     if (a < (b-c)):
#         print("Pred Correct:")
#         print(a)
#         print("Avg Correct:")
#         print(b)

#     a = temp.iloc[0]['regionsCorrect']
#     b = v_temp.regionsCorrect.mean()
#     c = v_temp.regionsCorrect.std()
#     if (a < (b-c)):
#         print("Reg Correct:")
#         print(a)
#         print("Avg Correct:")
#         print(b)

#     a = temp.iloc[0]['levelCorrect']
#     b = v_temp.levelCorrect.mean()
#     c = v_temp.levelCorrect.std()
#     if (a < (b-c)):
#         print("Lvl Correct:")
#         print(a)
#         print("Avg Correct:")
#         print(b)

#     a = temp.iloc[0]['ghostsCorrect']
#     b = v_temp.ghostsCorrect.mean()
#     c = v_temp.ghostsCorrect.std()
#     if (a < (b-c)):
#         print("Ghost Correct:")
#         print(a)
#         print("Avg Correct:")
#         print(b)

#     a = temp.iloc[0]['comparisonsCorrect']
#     b = v_temp.comparisonsCorrect.mean()
#     c = v_temp.comparisonsCorrect.std()
#     if (a < (b-c)):
#         print("Comp Correct:")
#         print(a)
#         print("Avg Correct:")
#         print(b)

    a = temp.iloc[0]['timeTakenMain_All']
    b = v_temp.timeTakenMain_All.mean()
    c = v_temp.timeTakenMain_All.std() * 3
    if (a < (b-c)):
        print(str(pID) + "......................................................................")
        sus_list.append(pID)
        print("Main Task:")
        print(a / 60)
        print("Avg Time:")
        print(b / 60)

    a = temp.iloc[0]['timeTakenMain_AgentsOnly']
    b = v_temp.timeTakenMain_AgentsOnly.mean()
    c = v_temp.timeTakenMain_AgentsOnly.std() * 3
    if (a < (b-c)):
        print(str(pID) + "......................................................................")
        sus_list.append(pID)
        print("Agents:")
        print(a / 60)
        print("Avg Time:")
        print(b / 60)

    a = temp.iloc[0]['timeTaken_Demographics']
    b = v_temp.timeTaken_Demographics.mean()
    c = v_temp.timeTaken_Demographics.std() * 3
    if (a < (b-c)):
        print(str(pID) + "......................................................................")
        sus_list.append(pID)
        print("Demo:")
        print(a / 60)
        print("Avg Time:")
        print(b / 60)

#     a = temp.iloc[0]['timeTaken_AI']
#     b = v_temp.timeTaken_AI.mean()
#     c = v_temp.timeTaken_AI.std()
#     if (a < (b-c)):
#         print("AI:")
#         print(a)
#         print("Avg Time:")
#         print(b)

    a = temp.iloc[0]['time_TOTAL']
    b = v_temp.time_TOTAL.mean()
    c = v_temp.time_TOTAL.std() * 4
    if (a < (8*60)):
        print(str(pID) + "......................................................................")
        sus_list.append(pID)
        print("All:")
        print(a / 60)
        print("Avg Time:")
        print(b / 60)
    elif (a > (b+c)):
        print(str(pID) + "......................................................................")
        sus_list.append(pID)
        print("All:")
        print(a / 60)
        print("Avg Time:")
        print(b / 60)

    vis_l = taskDF[taskDF['VisCond']==temp.iloc[0]['visuals']]
    vis_l = vis_l[vis_l['ExperimentVersion']==temp.iloc[0]['ExperimentVersion']]
    temp_L = taskDF[taskDF['ParticipantPublicID']==pID]
#     temp_L = temp_L[temp_L['ExperimentVersion']==temp.iloc[0]['ExperimentVersion']]

    a = len(temp_L[temp_L['Response']=='VIDEO PLAYING EVENT FIRED'])
    print(temp_L['Response'].unique())
    vidDF = vis_l[vis_l['Response']=='VIDEO PLAYING EVENT FIRED']
    b = len(vidDF)/len(vidDF.ParticipantPrivateID.unique())
    if (a == 0):
        print("Vid played: " + str(a))
        print("Avg vid played: " + str(b))
        sus_list.append(pID)
        
print(len(sus_list))

[None 'VIDEO STARTED' 'VIDEO PLAYING EVENT FIRED'
 'VIDEO TIMEUPDATE EVENT FIRED'
 'ADJUSTED START TIME based on TIMEUPDATE EVENT' 'Power Pill'
 'Ms. Pacman Loses a Life' 'Normal Ghost' 'Ghosts Turn Dark Blue'
 'VIDEO ENDED EVENT FIRED' 'Go Left' 'Go Up' '3&4' '2' 'Some Of The Time'
 'Two or More are Tied' 'Somewhat True' 'Very True' 'Neutral' 'Go Right'
 '3' '4' 'Red Ghost' 'Orange Ghost' 'Agent Thompson' 'Somewhat Confident'
 'Although none of the graphs showed that the ias were able to capture all the power pills, I think that with a little more training these models could complete the task.'
 'Training progress statistics' 'Agent Jackson'
 'It was the only AI that shown an approach to the ghosts while they were blue'
 "Videos of agent thompson's behavior when ghosts are blue "
 'In their statistics, the average of their steps was higher and has a lower standard deviation'
 'The mean and standard deviation of steps and scores obtained'
 'It is the agent that shows the highest consum

[None 'VIDEO STARTED' 'VIDEO PLAYING EVENT FIRED'
 'VIDEO TIMEUPDATE EVENT FIRED'
 'ADJUSTED START TIME based on TIMEUPDATE EVENT' 'Power Pill'
 'Ms. Pacman Loses a Life' 'Normal Ghost' 'Ghosts Turn Dark Blue'
 'VIDEO ENDED EVENT FIRED' 'Go Left' 'Go Right' '4' '1' 'Yes, Every Time'
 '2' 'I Do Not Know' 'Somewhat False' 'Somewhat True' 'Go Down' '1&2'
 'I Cannot Tell' 'Some Of The Time' 'Orange Ghost' 'Neutral' 'Very False'
 'Agent Jones' 'Somewhat Confident'
 "I had the impression it was part of Agent Jones' tactic to keep the ghosts blue as long as possible"
 "I'm not sure"
 'It follows that if agent Jones is actively looking for the pills, they want to get rid of the ghosts quickly'
 'No idea' 'Agent Smith' 'Not Confident'
 'It felt like Agent Smith is less likely to take risky turns'
 'It seemed to me like Agent Jones was looking to quickly get to the pills'
 "I don't know" 'Using the tunnels on the sides of the maze ' 'Black'
 'Unlikely' 'Increase by a lot' 'Very Engaging']
[None 

[None 'VIDEO STARTED' 'VIDEO PLAYING EVENT FIRED'
 'VIDEO TIMEUPDATE EVENT FIRED'
 'ADJUSTED START TIME based on TIMEUPDATE EVENT' 'Power Pill'
 'Ms. Pacman Loses a Life' 'Normal Ghost' 'Ghosts Turn Dark Blue'
 'VIDEO ENDED EVENT FIRED' 'Go Up' 'I Do Not Know' 'Go Down' '2' '3'
 'Yes, Every Time' 'Neutral' 'Somewhat True' 'Somewhat False' 'Go Left'
 'Go Right' '1&2' '4' 'Some Of The Time' 'Agent Smith'
 'Completely Confident'
 'Because Agent Smith clearly follows that sort of strategy, of consuming those pills early in terms of level completion time and not save them up.'
 'I am not sure about this, to be honest.' 'Somewhat Confident'
 'I cannot be certain in the end, but it seems that Agent Smith did eat more ghosts in the video excerpts (and was about to eat some more in some of them). Apart from that, Agent Smith was rather keen on consuming Power Pills early on, which might mean he was also going for the ghosts and did not want to miss any chances.'
 'Information about scoring woul

[None 'VIDEO STARTED' 'VIDEO PLAYING EVENT FIRED'
 'VIDEO TIMEUPDATE EVENT FIRED'
 'ADJUSTED START TIME based on TIMEUPDATE EVENT' 'Power Pill'
 'Ms. Pacman Loses a Life' 'Normal Ghost' 'Ghosts Turn Dark Blue'
 'VIDEO ENDED EVENT FIRED' 'Go Up' 'Go Down' '4' '3' 'Yes, Every Time' '2'
 'I Do Not Know' 'Neutral' 'Somewhat False' 'Somewhat True' 'Go Right' '1'
 'Some Of The Time' 'Agent Smith' 'Completely Confident'
 'Based off the chart bar which showed that this AI compared to the other collects them much ealier and quicker'
 'Maybe show the full clip of the AI not just some situations'
 'Agent Jones' 'Not Confident'
 'Judging by the chart bubles again, I think they were pretty similar in amount just by first look, so It was hard to tell which has more.'
 'Numbers telling exactly how many they did in each game'
 "It's obvious judging by the charts that this AI is able to survive longer and saves on powerups for later"
 'exact time telling how much each survived in each game'
 'This AI h

[None 'VIDEO STARTED' 'VIDEO PLAYING EVENT FIRED'
 'VIDEO TIMEUPDATE EVENT FIRED'
 'ADJUSTED START TIME based on TIMEUPDATE EVENT' 'Power Pill'
 'Ms. Pacman Loses a Life' 'Normal Ghost' 'Ghosts Turn Dark Blue'
 'VIDEO ENDED EVENT FIRED' 'Go Up' 'Go Right' '3' '1' 'Some Of The Time'
 '2' 'Orange Ghost' 'Two or More are Tied' 'Very False' 'Somewhat False'
 'Somewhat True' 'Go Left' 'Go Down' 'Agent Smith' 'Somewhat Confident'
 'It appears as though Agent Smith is more reliable playing this game.'
 'I would prefer to see the videos unblurred and in better quality.'
 "Agent Smith's movements were more confident and it looked as though the AI had a better understanding of how to play and the goals of the game."
 'Seeing a longer video of both of them playing.'
 "Agent Smith didn't lose a heart in the shown scenarios."
 'Seeing a longer and unblurred video of the gameplay.'
 'Agent Smith seems to be better at playing this game.'
 'Longer and unblurred video of the gameplay of both AIs.'
 'Us

...

[None 'VIDEO STARTED' 'VIDEO PLAYING EVENT FIRED'
 'VIDEO TIMEUPDATE EVENT FIRED'
 'ADJUSTED START TIME based on TIMEUPDATE EVENT' 'Power Pill'
 'Ms. Pacman Loses a Life' 'Normal Ghost' 'Ghosts Turn Dark Blue'
 'Go Left' 'Go Up' '3' '3&4' 'No, Never' 'Teal Ghost' 'Red Ghost'
 'Somewhat True' 'Neutral' '1' 'Go Down' 'Go Right' '4' 'Yes, Every Time'
 'Very True' 'Agent Thompson' 'Somewhat Confident'
 'It looks from the videos that Agent Thompson is preoccupied with eating hte power pills without care for strategy'
 'Seeing Agent Thompson go for the pill at a bad time in the scenarios'
 'Agent Smith' 'Completely Confident'
 'Agent Smith has a more strategic approach to eating power pills and then going for the ghosts'
 "More scenarios when the ghosts aren't already blue"
 'Agent Smith is good at eating blue ghosts' 'Longer scenario clips'
 'Agent Smith was more strategic in using the power pill at a good time.'
 'Longer scenario videos to watch Agent Smith succeed'
 'Using the tunnels on 

[None 'VIDEO STARTED' 'VIDEO PLAYING EVENT FIRED'
 'VIDEO TIMEUPDATE EVENT FIRED'
 'ADJUSTED START TIME based on TIMEUPDATE EVENT' 'Power Pill'
 'Ms. Pacman Loses a Life' 'Normal Ghost' 'Ghosts Turn Dark Blue' 'Go Up'
 'Go Left' '3' '1' 'Some Of The Time' 'I Do Not Know' 'Somewhat False'
 'Somewhat True' 'Very False' 'Go Down' 'I Cannot Tell' 'Yes, Every Time'
 'Very True' 'Neutral' 'Agent Jackson' 'Somewhat Confident'
 'Agent jackson scenarios showed multiple scenarios where the ghosts were scared'
 'The state and number of ghosts' 'Agent Jones'
 'There are only three ghosts in two scenarios and Jones is eating scared ghost in the other'
 'The number of ghosts visible' 'Completely Confident'
 'Agents jones had the most dots missing which showed that he was alive longer'
 'number of dots on screen' 'Not at All Confident'
 'More than three power pills appear in multiple scenaios'
 'more than three ghosts seem to be present at all times '
 'Using the tunnels on the sides of the maze ' 'B

[None 'VIDEO STARTED' 'VIDEO PLAYING EVENT FIRED'
 'VIDEO TIMEUPDATE EVENT FIRED'
 'ADJUSTED START TIME based on TIMEUPDATE EVENT' 'Power Pill'
 'Ms. Pacman Loses a Life' 'Normal Ghost' 'Ghosts Turn Dark Blue'
 'Go Down' 'I Do Not Know' '3' '4' 'Some Of The Time' 'Somewhat True'
 'Somewhat False' '1' 'Go Up' '3&4' '2' 'Neutral' 'Agent Jones'
 'Completely Confident'
 'Because the chart shows that Agent Jones gets more power pills on each turn, and gets them much quicker.'
 'Seeing how long it takes them on average to finish each level.'
 'Agent Jones had by far more eaten ghosts according to the chart.'
 'None.'
 'According to the chart, Jones typically plays for a longer amount of time.'
 'Agent Jackson' 'Not Confident'
 'I think that Jackson breaks up their use of power pills more.'
 'Knowing how often they try to use the strategy of preserving power pills for later lives.'
 'Using the tunnels on the sides of the maze ' 'Black' 'Likely'
 'Increase by a lot' 'Somewhat Engaging']
[None 

[None 'VIDEO STARTED' 'VIDEO PLAYING EVENT FIRED'
 'VIDEO TIMEUPDATE EVENT FIRED'
 'ADJUSTED START TIME based on TIMEUPDATE EVENT' 'Power Pill'
 'Ms. Pacman Loses a Life' 'Normal Ghost' 'Ghosts Turn Dark Blue' 'Go Up'
 'Go Left' 'Go Down' '3&4' '4' '3' 'Some Of The Time' 'I Do Not Know'
 'Neutral' 'Somewhat True' '1' 'Go Right' 'Agent Jones'
 'Somewhat Confident'
 'because it moves more around the map and around the areas that contain power pills'
 'i dont know' 'Agent Smith' 'I think he is closer to the ghosts'
 'Maybe the moviment around the map'
 'The number of steps he took in the game is bigger' "I don't know"
 'he takes less pills in the game according to the game bar chart' 'none'
 'Using the tunnels on the sides of the maze ' 'Black' 'Unlikely'
 'No change']
[None 'VIDEO STARTED' 'VIDEO PLAYING EVENT FIRED'
 'VIDEO TIMEUPDATE EVENT FIRED'
 'ADJUSTED START TIME based on TIMEUPDATE EVENT' 'Power Pill'
 'Ms. Pacman Loses a Life' 'Normal Ghost' 'Ghosts Turn Dark Blue'
 'Go Left' 'G

[None 'VIDEO STARTED' 'VIDEO PLAYING EVENT FIRED'
 'VIDEO TIMEUPDATE EVENT FIRED'
 'ADJUSTED START TIME based on TIMEUPDATE EVENT' 'Power Pill'
 'Ms. Pacman Loses a Life' 'Normal Ghost' 'Ghosts Turn Dark Blue' 'Go Up'
 'Go Right' '3' '4' 'Some Of The Time' 'I Do Not Know' 'Somewhat True' '1'
 'Go Left' 'I Cannot Tell' 'Agent Jones' 'Somewhat Confident'
 'Per game info chart showed Agent Jones eating pills faster than Agent Jackson'
 "As per Heatmap of Visits Per Coordinate Agent Jones is the one covering most areas and reaching out to all the pills. Agent Jackson doens't seem to eat all pills according to Game Bar Per Game Info Chart"
 'By comparing the Game Bar Per Info Chart'
 'Chart shows that Agent Jones eats more ghosts according to the color metric in Game Bar Info'
 'Bars for each game durationn on Game Bar Chart metrics shows that on average Agent Jones has the longest gameplay'
 'Agent Jones is more likely to escape from ghosts and stay alive due to is bigger covering of the g

In [34]:
############################################# STEP TWO #############################################
for pID in sus_list:
    tT_2 = tempT[tempT['ParticipantPublicID']==pID]
    sT_2 = tempS[tempS['ParticipantPublicID']==pID]
    
    # check percent correct
    if (sT_2.totPerc.unique() < 110):
        # check failed attention checks
        attn = tT_2.query('(display=="Attn1" | display=="Attn2" | display=="AttnCheck") & ZoneType=="response_button_text"')
#         print(attn['Correct'].sum())
        if (attn['Correct'].sum() < 2):
#             print("!! Missed 3 attention checks !!")
#             print(pID)
#             print(len(attn))
            assert(True)
        else:
            print("\n" + pID)
            sus_list.remove(pID)
            if (sT_2.totPerc.unique()[0] < 40):
                print("perc: " + str(sT_2.totPerc.unique()[0]))

            # check time on main task and agents only
            if (sT_2.timeTakenMain_All.unique()[0] < 1000):
                print("All " + str(sT_2.timeTakenMain_All.unique()[0]/60))
            if (sT_2.timeTakenMain_AgentsOnly.unique()[0] < 400):
                print("Agents " + str(sT_2.timeTakenMain_AgentsOnly.unique()[0]/60))

            # check for repetition in button
            reg = tT_2.query('display=="Regions" & ZoneType=="response_button_text"')
            if (len(reg.Response.unique()) < 2):
                print("......................................regions")
                print(pID)
                print(reg.Response)
        #     if (len(reg.ZoneName.unique()) < 3):
        #         print(reg.ZoneName)
            pred = tT_2.query('display=="Prediction" & ZoneType=="response_button_text"')
            if (len(pred.Response.unique()) < 2):
                print("......................................predictions")
                print(pID)
                print(pred.Response)
        #     if (len(pred.ZoneName.unique()) < 4:
        #         print(pred.ZoneName)
            likert = tT_2.query('display=="likerts" & ZoneType=="response_button_text"')
            if (len(likert.Response.unique()) < 2):
                print("......................................likerts")
                print(likert.Response)
        #     if (len(likert.ZoneName.unique()) < 3):
        #         print(likert.ZoneName)

            # check failed attention checks
#             attn = tT_2.query('(display=="Attn1" | display=="Attn2" | display=="AttnCheck") & ZoneType=="response_button_text"')
#             print(attn['Correct'].sum())
#             if (attn['Correct'].sum() < 4):
#                 print("!! Missed 3 attention checks !!")
#                 print(pID)
#                 print(len(attn))

            # look at text responses
            print(">>>>>>>>>>>>>>>>>>>>>>>>>")
            print(pID)
            text = tT_2.query('ZoneType=="response_text_area"')
            print(text.Response)
            
print(len(sus_list))

3


/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  import sys


In [35]:
print(len(sus_list))
tempS = slimDF[~slimDF.ParticipantPublicID.isin(sus_list)]
print(len(tempS))
tempT = taskDF[~taskDF.ParticipantPublicID.isin(sus_list)]
print(len(tempT))

3
246
74912


In [36]:
tempDir = '/Volumes/Britt_SSD/ReDoEvals3/'
tempTopDir = os.path.join(tempDir, 'UserStudy')
if not os.path.isdir(tempTopDir):
    os.mkdir(tempTopDir)
tempTopDir = os.path.join(tempDir, 'UserStudy')

# Write Short Task DF to parquet:
filePath = os.path.join(tempTopDir, 'GP_Main_Short_Sussed.parquet')
tempS.to_parquet(path=filePath,compression='brotli')